<a href="https://colab.research.google.com/github/Umhyunbin/review_analysis/blob/master/playstore_review_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
# for scraping app info and reviews from Google Play
from google_play_scraper import app, Sort, reviews_all
from collections import Counter

In [ ]:
## Get app names and app IDs
app_names = input("원하는 앱 이름을 입력하세요: ")
app_ids = input("앱 아이디를 입력하세요(url에서 확인 가능합니다): ")

In [ ]:
# App all reviews

"""
first parameter -> app_ids(in url)
lang, country -> change for korean
"""
result_all = pd.DataFrame(columns = ['리뷰', '평점', '공감수', '작성일자'])
for i in range(1, 3):
    result = reviews_all(
        app_ids,
        sleep_milliseconds=0, # defaults to 0
        lang='ko', # defaults to 'en'
        country='kr', # defaults to 'us'
        sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
        filter_score_with = i # defaults to None(means all score) -> only scrap negative review
    )

    #df_review = pd.DataFrame(result, columns = ['리뷰시간', '리뷰', '답변시간', '답변', '리뷰버전', '리뷰ID', '별점', '좋아요', '유저이미지', '유저이름'])
    df_review = pd.DataFrame(result)
    df_review.drop(['reviewId', 'userName', 'userImage', 'reviewCreatedVersion', 'replyContent', 'repliedAt'], axis = 1, inplace = True)
    df_review.columns = ['리뷰', '평점', '공감수', '작성일자']

    result_all = pd.concat([result_all, df_review])

'''
print(result_all.head())
print(result_all.info())
print(result_all.describe())
print(result_all.isna().sum())
print(result_all[result_all['리뷰'].isna()])
'''

# not exist review -> drop 
result_all.dropna(axis = 0,inplace = True)

# want to show 5 years
result_all['년도'] = result_all['작성일자'].apply(lambda x: x.year)
result_all = result_all[result_all['년도'] > 2017]
result_all = result_all.astype({'평점' : 'int', '공감수' : 'int'})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from konlpy.tag import Okt
from collections import defaultdict

okt = Okt()
# ranks.nl -> korean stopword
s = pd.read_csv("/content/drive/MyDrive/리뷰분석/불용어사전.csv")

stop_words = set(str(s.columns).split())
# {year1 : [word1, ...]}
li_nouns = defaultdict(list)

for i in range(len(result_all)):
    review = result_all.iloc[i, 0]
    year = result_all.iloc[i, 4]
    # normalize
    review_normalized = okt.normalize(review)
    line_nouns = okt.nouns(review_normalized)
    li_nouns[year] += [noun for noun in line_nouns if len(noun) > 1 and noun not in stop_words]

count_nouns = []
for y in range(2018, 2023):
    # counter -> {word1 : freq1, ...}
    counter = dict(Counter(li_nouns[y]))
    # count_nouns -> [(word1, freq1, year1), ...]
    count_nouns += list(zip(list(counter.keys()), list(counter.values()), [y] * len(counter)))

count_nouns = list(map(list, count_nouns))
# sort by word, year
count_nouns_sorted = sorted(count_nouns, key = lambda x: (x[0], x[2]))
result_df = pd.DataFrame(count_nouns_sorted, columns = ['단어', '빈도', '년도'])
result_df.to_csv('/content/drive/MyDrive/리뷰분석/년도별_단어_빈도.csv', index = False)